# Data 처리

In [1]:
#log_file = './log_fastapi_vllm_mistral7b0.2_50users.out'
#log_file = './log_fastapi_trrt_mistral7b0.2_50users.out'
log_file = './log_fastapi_vllm_mistral7b0.2_100users.out'

In [2]:
import pandas as pd
from datetime import datetime

# Using plotly as graph backend
pd.options.plotting.backend = "plotly"

log_df = pd.read_csv(log_file,
                     delimiter=' \|\| ',
                     names=['epoch_time', 'user_Id', 'message1', 'message2'],
                     engine='python') # for surpressing warning msg
# sorted_df = log_df.sort_values(by=['user_Id', 'epoch_time'])

userIds: list[str] = log_df["user_Id"].unique()

caculated_df_columns = ['userId', 'epoch_time', 'start_time', 'end_time', 'latency', 'ttft', 'tpot_avg', 'tpot_std', 'num_of_tokens', 'tokens_per_sec']
caculated_df = pd.DataFrame(columns=caculated_df_columns)

for userId in userIds:
    df = log_df[log_df['user_Id'] == userId]

    start_time = df.iloc[1]['epoch_time']
    end_time = df.iloc[-1]['epoch_time']

    latency = (end_time - start_time) / 1000

    if (df.size < 3):
        continue

    try:
        ttft = df.iloc[2]['epoch_time'] - df.iloc[1]['epoch_time']
    except:
        continue
    time_differences = df['epoch_time'].diff()[1:]
    tpot_avg = time_differences.mean()
    tpot_std = time_differences.std()

    num_of_tokens = len(df[df['message1'].str.contains('Token received :') == True].index)

    data = {'userId': userId, 'epoch_time': end_time, 'start_time': start_time, 'end_time': end_time, 'latency': latency, 'ttft': ttft, 'tpot_avg': tpot_avg, 'tpot_std': tpot_std, 'num_of_tokens': num_of_tokens, 'tokens_per_sec': num_of_tokens / latency}

    caculated_df.loc[len(caculated_df)] = data

caculated_df = caculated_df.sort_values(by=['epoch_time'])

# 램프업 데이터
rampup_sequence_df = log_df[log_df['message1'].str.contains('CurrentUserCount') == True][['epoch_time', 'message2']]
rampup_sequence_df = rampup_sequence_df.sort_values(by=['epoch_time'])
rampup_sequence_df = rampup_sequence_df.rename(columns={'message2': 'UserCount'})
rampup_sequence_df['UserCount'] = rampup_sequence_df['UserCount'].map(lambda x: int(x))

caculated_df['Datetime(KST)'] = caculated_df['epoch_time'].map(lambda x: datetime.fromtimestamp(x / 1000))
rampup_sequence_df['Datetime(KST)'] = rampup_sequence_df['epoch_time'].map(lambda x: datetime.fromtimestamp(x/1000))

# 시간 흐름에 따른 토큰 생성량
tokens_df = log_df[log_df['message1'].str.contains('Token received :') == True]
tokens_df['Datetime(KST)'] = tokens_df['epoch_time'].map(lambda x: datetime.fromtimestamp(x / 1000).replace(microsecond=0))

tokens_df = tokens_df.groupby('Datetime(KST)')['epoch_time'].count().reset_index()
tokens_df = tokens_df.rename(columns={'epoch_time': 'Tokens'})


/var/folders/09/nrd0f7kd5rj1ds8c7s02dwwr16q1m1/T/ipykernel_79779/3870826162.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tokens_df['Datetime(KST)'] = tokens_df['epoch_time'].map(lambda x: datetime.fromtimestamp(x / 1000).replace(microsecond=0))


# 결과 출력

In [3]:

rampup_sequence_df.plot.line(
                        x='Datetime(KST)',
                        y='UserCount')

# 램프업, 토큰 생성량, 지연 시간
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig_rampup = px.line(rampup_sequence_df, x = 'Datetime(KST)', y = 'UserCount')
fig_tokenspeed = px.scatter(tokens_df, x = 'Datetime(KST)', y = 'Tokens', trendline='lowess', trendline_options=dict(frac=0.5))
fig_latency = px.scatter(caculated_df, x = 'Datetime(KST)', y = 'latency', trendline='lowess', trendline_options=dict(frac=0.5))

fig_total = make_subplots(rows=3, cols=1,
                          shared_xaxes = True)
                          #subplot_titles=('User Count', 'Token Generation Speed', 'Latency'))


for d in fig_rampup.data:
    fig_total.add_trace((go.Line(x = d['x'], y = d['y'], name = 'User Count')), row = 1, col = 1)

for d in fig_tokenspeed.data:
    fig_total.add_trace((go.Scatter(x = d['x'], y = d['y'], name = 'Tokens/s', mode='markers')), row = 2, col = 1)

for d in fig_latency.data:
    fig_total.add_trace((go.Scatter(x = d['x'], y = d['y'], name = 'Latency', mode='markers')), row = 3, col = 1)

fig_total.update_layout(hovermode = 'x unified', height=800)
fig_total.update_xaxes(showspikes=True, spikethickness=0.5)
fig_total.update_yaxes(showspikes=True, spikethickness=0.5)
fig_total.update_traces(marker_size=2)

fig_total.show()


/Users/1111959/Projects/LLM/LLM-serving/loadtest-client/.venv/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [4]:
fig_ttft = px.scatter(caculated_df, x = 'Datetime(KST)', y = 'ttft', trendline='lowess', trendline_options=dict(frac=0.5))
fig_tpot = px.scatter(caculated_df, x = 'Datetime(KST)', y = 'tpot_avg', trendline='lowess', trendline_options=dict(frac=0.5))

fig_tpot.add_trace(go.Scatter(x = caculated_df['Datetime(KST)'], y = caculated_df['tpot_std'], name='tpot_std'))

fig_total2 = make_subplots(rows=3, cols=1,
                          shared_xaxes = True)

for d in fig_rampup.data:
    fig_total2.add_trace((go.Line(x = d['x'], y = d['y'], name = 'User Count')), row = 1, col = 1)

for d in fig_ttft.data:
    fig_total2.add_trace((go.Scatter(x = d['x'], y = d['y'], name = 'TTFT', mode='markers')), row = 2, col = 1)

for d in fig_tpot.data:
    fig_total2.add_trace((go.Scatter(x = d['x'], y = d['y'], name = 'TPOT', mode='markers')), row = 3, col = 1)

fig_total2.update_layout(hovermode = 'x unified', height=800)
fig_total2.update_xaxes(showspikes=True, spikethickness=0.5)
fig_total2.update_yaxes(showspikes=True, spikethickness=0.5)
fig_total2.update_traces(marker_size=2)

fig_total2.show()

/Users/1111959/Projects/LLM/LLM-serving/loadtest-client/.venv/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


